In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [26]:
def get_buy_webpage(city, 
                    state, 
                    pg_num, 
                    htype=[
                        'house',
                        'multi-family',
                    ]):
    
    overhead = 'https://www.trulia.com'
    dangle = 'for_sale'

    city = city.title()\
               .replace(' ', '_')
    state = state.upper()

    dict_alias = {
        'house': 'SINGLE-FAMILY_HOME',
        'condo': 'APARTMENT,CONDO,COOP',
        'townhouse': 'TOWNHOUSE',
        'multi-family': 'MULTI-FAMILY',
        'land': 'LOT%7CLAND',
        'mobile/manufactured': 'MOBILE%7CMANUFACTURED',
        'other': 'UNKNOWN',
    }

    aliases = [dict_alias[h] for h in htype]
    houses = ','.join(aliases)

    webpage = f'{overhead}/{dangle}/{city},{state}/{houses}_type/{pg_num}_p/'
    return webpage


In [62]:
url = get_buy_webpages('philadelphia', 'pa', 1)

In [82]:
def get_buy_apt_urls_per_page(city,
                              state,
                              pg_num,
                              htype=['house', 
                                     'multi-family']):

    webpage = get_buy_webpage(city, state, pg_num, htype)
    
    # Here we added User-Agent to the header of our request 
    # It is because sometimes the web server will check the
    # different fields of the header to block robot scrapers
    # User-Agent is the most common one because it is specific 
    # to your browser.
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(webpage, headers=headers)
    results = response.content

    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
        apt_tags = soup.find_all('div', class_='PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA')
        apt_link_tags = [tag.find('a') for tag in apt_tags]
        apt_urls = [tag['href'] for tag in apt_link_tags]
        return len(apt_urls)
        

In [83]:
get_buy_apt_urls_per_page('philadelphia', 'pa', 1)

30